# Study Tutorial



## Installation and configuration
The installation and configuration process is discussed in detail in the `BasicTutorial`. Of course, if we don't already have `osparc` installed we need to do so and we need to setup the `osparc.Configuration`:

In [ ]:
import os
import importlib.util
if importlib.util.find_spec('osparc') is None:
    ! pip install osparc
from osparc import Configuration
cfg: Configuration = Configuration(
    host=os.environ["OSPARC_API_HOST"],
    username=os.environ["OSPARC_API_KEY"],
    password=os.environ["OSPARC_API_SECRET"]
)

# this tutorial is compatible with version > 0.5.0
assert tuple(map(int, osparc.__version__.split("."))) > (0, 5, 0)

## Study Workflow

Studies can connect computational resources as well as pre- and postprocess data. Studies mmust be created through the UI (where you generated the `OSPARC_API_KEY` and the `OSPARC_API_SECRET`). Once a study has been created via the UI one can interact with it (modify, run, read and write data to it etc.) using the `osparc` python package. Here we will demonstrate the typical workflow. As an illustration we consider a simple study containing a single sleeper solver.


In [ ]:
import time
from pathlib import Path
from zipfile import ZipFile
from tempfile import TemporaryDirectory

import osparc

# Get the ID of a study under the study information
SLEEPER_STUDY_ID: str = "fcad50ac-253e-11ee-9414-02420a0f071c"

with osparc.ApiClient(cfg) as api_client:

    studies_api: osparc.StudiesApi = osparc.StudiesApi(api_client)
    study: osparc.Study = studies_api.get_study(SLEEPER_STUDY_ID)


    job: osparc.Job = studies_api.create_study_job(study_id=study.id, 
                                                   job_inputs=osparc.JobInputs({
                                                       "X": 1
                                                   }))

    status: osparc.JobStatus = studies_api.start_study_job(study.id, job_id=job.id)
    while not status.stopped_at:
        time.sleep(3)
        status = studies_api.inspect_study_job(study.id, job.id)
        print("Solver progress", f"{status.progress}/100", flush=True)

    outputs: osparc.JobOutputs = studies_api.get_study_job_outputs(study_id=study.id, job_id=job.id)

    print(f"Job {outputs.job_id} got these results:")
    for output_name, result in outputs.results.items():
        print(output_name, "=", result)

    logfile_path: str = studies_api.get_study_job_output_logfile(study.id, job.id)
    zip_path = Path(logfile_path)

    with TemporaryDirectory() as tmp_dir:
        with ZipFile(f"{zip_path}") as fzip:
            fzip.extractall(tmp_dir)
        logfiles = list(Path(tmp_dir).glob("*.log*"))
        print("Unzipped", logfiles[0], "contains:\n", logfiles[0].read_text())


The script above

1. Gets the sleeper study
2. Submits a job to the sleeper study (asking it to sleep for `X=1` second)
3. When the execution completes, it checks the outputs
4. The logs are downloaded and extracted


## References

- [osparc API python client] documentation
- [osparc API] documentation

[osparc API python client]:https://itisfoundation.github.io/osparc-simcore-python-client
[osparc API]:https://api.osparc.io/doc